In [2]:
import findspark 
findspark.init() 

import pyspark
from delta import *
import pandas as pd
from pyspark.sql.functions import col

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
spark = pyspark.sql.SparkSession.builder.master("spark://spark-master:7077").appName("spark_kafka").config("spark.sql.streaming.metricsEnabled", "true").getOrCreate()

sc = spark.sparkContext
sc.setLogLevel('ERROR')

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /opt/spark/cache
The jars for the packages stored in: /opt/spark/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.mysql#mysql-connector-j added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-616921c3-c50b-44c2-855e-3a075efb39f2;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.ap

23/03/17 19:12:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/17 19:12:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
spark.sql("show databases").toPandas()

,namespace
0,bronze
1,default
2,silver
3,test_db


In [8]:
spark.sql("show tables in test_db").toPandas()

,namespace,tableName,isTemporary
0,test_db,spark_kafka,False
1,test_db,test_table,False
2,test_db,weather_test_v1,False


In [5]:
# spark.sql("drop table test_db.spark_kafka").toPandas()

In [6]:
spark.sql("create database if not exists bronze")

DataFrame[]

In [7]:
# topic = "test-topic-1"
topic = "weather_test_v1"
maxOffsetsPerTrigger = 100

df = spark\
    .readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", "kafka-broker:9092")\
    .option("subscribe", topic)\
    .option("maxOffsetsPerTrigger", maxOffsetsPerTrigger) \
    .option("startingOffsets", "earliest") \
    .load()

In [7]:
# df.selectExpr("cast(value as string) as payload", "*").limit(2).toPandas()

In [8]:
trigger_interval = "60 seconds"
queryName = "weather_test_v1"
tableName = "weather_test_v1"

df\
.selectExpr("key", "topic", "partition", "offset", "timestamp", "timestampType", "cast(value as string) as payload") \
.writeStream \
.trigger(processingTime = trigger_interval) \
.queryName(queryName) \
.format("delta") \
.outputMode("append") \
.option("checkpointLocation", f"/opt/spark/work-dir/odp_intra_storage/spark/datalake/bronze.db/{tableName}/_ckpt") \
.toTable(f"bronze.{tableName}")

In [9]:
spark.sql("select payload from bronze.weather_test_v1 limit 2").toPandas()

,payload
0,"{""data"": {""location"": ""Naperville"", ""time"": ""Friday 3:00\u202fPM"", ""info"": ""Snow showers"", ""temperature"": ""30"", ""precipitation"": ""1%"", ""humidiy"": ""54%"", ""wind"": ""9 mph""}}"
1,"{""data"": {""location"": ""Naperville"", ""time"": ""Friday 3:00\u202fPM"", ""info"": ""Snow showers"", ""temperature"": ""30"", ""precipitation"": ""1%"", ""humidiy"": ""54%"", ""wind"": ""9 mph""}}"


In [22]:
spark.sql("select current_timestamp - INTERVAL 10 minutes").toPandas()

/opt/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,current_timestamp() - INTERVAL '10' MINUTE
0,2023-02-24 22:58:39.676830


In [7]:
spark.sql("OPTIMIZE test_db.spark_kafka").toPandas()

,path,metrics
0,file:/opt/spark/work-dir/odp_intra_storage/spa...,"(1, 15740, (874746, 874746, 874746.0, 1, 87474..."


In [11]:
spark.sql("set spark.databricks.delta.retentionDurationCheck.enabled = false")
spark.sql("SET spark.databricks.delta.vacuum.parallelDelete.enabled = true")
spark.sql("VACUUM test_db.spark_kafka RETAIN 0 HOURS").toPandas()

Deleted 0 files and directories in a total of 1 directories.


,path
0,file:/opt/spark/work-dir/odp_intra_storage/spark/datalake/test_db.db/spark_kafka


In [16]:
select_expr = ["get_json_object(payload, '$.data.location') as location",
 "get_json_object(payload, '$.data.time') as time",
 "get_json_object(payload, '$.data.info') as info",
 "get_json_object(payload, '$.data.temperature') as temperature",
 "get_json_object(payload, '$.data.precipitation') as precipitation",
 "get_json_object(payload, '$.data.humidiy') as humidity",
 "get_json_object(payload, '$.data.wind') as wind",
 "topic as kafka_topic",
 "partition as kafka_partition",
 "offset as kafka_offset",
 "cast(timestamp as timestamp) as kafka_timestamp",
 "cast(timestamp as date) as kafka_date", 
 "current_timestamp as load_time"
]

In [17]:
spark.table("bronze.weather_test_v1").selectExpr(select_expr).orderBy(col("kafka_offset").desc()).limit(2).toPandas()

/opt/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/opt/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,location,time,info,temperature,precipitation,humidity,wind,kafka_topic,kafka_partition,kafka_offset,kafka_timestamp,kafka_date,load_time
0,Naperville,Friday 5:00 PM,Mostly cloudy,28,68%,59%,9 mph,weather_test_v1,0,262,2023-02-24 23:00:29.735,2023-02-24,2023-02-24 23:01:50.568526
1,Naperville,Friday 4:00 PM,Mostly cloudy,28,68%,59%,9 mph,weather_test_v1,0,261,2023-02-24 22:59:58.830,2023-02-24,2023-02-24 23:01:50.568526


In [31]:
spark.sql("create database if not exists silver")

DataFrame[]

In [15]:
spark.sql("drop table if exists silver.weather_test")

DataFrame[]

In [18]:
spark.sql("""
create table silver.weather_test 
(
location string,
time string,
info string,
temperature string,
precipitation string,
humidity string,
wind string,
kafka_topic string,
kafka_partition int,
kafka_offset bigint,
kafka_timestamp timestamp,
kafka_date date,
load_time timestamp
)
using delta
;
""").toPandas()

""


In [19]:
trigger_interval = "60 seconds"
queryName = "silver_weather_test"
tableName = "weather_test"

spark.readStream \
.format("delta") \
.table("bronze.weather_test_v1") \
.selectExpr(select_expr) \
.writeStream \
.trigger(processingTime = trigger_interval) \
.queryName(queryName) \
.format("delta") \
.outputMode("append") \
.option("checkpointLocation", f"/opt/spark/work-dir/odp_intra_storage/spark/datalake/silver.db/{tableName}/_ckpt") \
.toTable(f"silver.{tableName}")

In [21]:
spark.sql("""
select 
kafka_timestamp
, max(temperature) as temperature
, max(cast(replace(precipitation, '%', '') as int)) as precipitation
, max(cast(replace(humidity, '%', '') as int)) as humidity
, max(cast(replace(wind, ' mph', '') as int)) as wind
from silver.weather_test 
where 1=1
and location = 'Naperville'
group by 1
;
""").limit(2).toPandas()

/opt/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,kafka_timestamp,temperature,precipitation,humidity,wind
0,2023-02-24 21:42:22.764,30,1,54,9
1,2023-02-24 21:42:33.751,30,1,54,9


In [11]:
spark.sql("""
select 
kafka_timestamp
, max(temperature) as temperature
--, max(cast(replace(precipitation, '%', '') as int)) as precipitation
--, max(cast(replace(humidity, '%', '') as int)) as humidity
--, max(cast(replace(wind, ' mph', '') as int)) as wind
from silver.weather_v1 
where 1=1
--and location = 'Naperville'
--and kafka_timestamp > (current_timestamp - interval 60 minutes)
group by 1
order by 1 desc
;
""").limit(2).toPandas()

/opt/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,kafka_timestamp,temperature
0,2023-03-16 05:57:42.464,40
1,2023-03-16 05:56:41.647,40
